In [56]:
# pip install watermark
%load_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,sklearn -g 
#,statsmodels

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.6.7
IPython 7.4.0

numpy 1.16.2
scipy 1.2.1
pandas 0.24.2
matplotlib 3.0.3
sklearn 0.20.3

compiler   : GCC 8.2.0
system     : Linux
release    : 4.9.93-boot2docker
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   :


In [57]:
from __future__ import division, print_function
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import math
import datetime
import re
import sys

In [58]:
COMPARATOR_AUTHOR = "Anton Tushev" 
COMPARATOR_VERSION = "v.11"
PATH_TO_DATA = '/notebooks/pinda/data'
#PATH_TO_STRUCG = '/notebooks/pinda/data/alcanc26_03_2019'
PATH_TO_STRUCG = '/notebooks/pinda/data/test'
PATH_TO_REPORTS = '/notebooks/pinda/reports'
FILE_NAME_DATA1 = 'TX04_fixed_v1.csv'
SD_RACK_NAMES = ['tx04']
FILE_NAME_TEMP1 = 'TX04_@progress_bkup.csv'
#FILE_NAME_DATA1 = 'test.csv'
IGNORE_CHART = ['@SIMD']
SKIP_EQUAL = True
SKIP_BLOCK_NEXST = True
SKIP_SIGNAL_NFND = True
SKIP_TYPE_STR = True
SKIP_CHART_NFND = True
COMPARATOR_README = "===================================================================================\
\r\nAdvanced comparator SimadynD <=> TDC software version: %s \r\nAuthor: %s \
\r\nComparison result made from %s StrucG folder \
\r\nTDC data file is '%s' \
\r\nList of SimadynD subfolders for parsing is %s \
\r\nList of ignored charts is %s \
\r\n===================================================================================\
"%(COMPARATOR_VERSION,COMPARATOR_AUTHOR,PATH_TO_STRUCG,FILE_NAME_DATA1,SD_RACK_NAMES,IGNORE_CHART)
print(COMPARATOR_README)

Advanced comparator SimadynD <=> TDC software version: v.11 
Author: Anton Tushev 
Comparison result made from /notebooks/pinda/data/test StrucG folder 
TDC data file is 'TX04_fixed_v1.csv' 
List of SimadynD subfolders for parsing is ['tx04'] 
List of ignored charts is ['@SIMD'] 


In [59]:
#Load TDC data
data_a1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_NAME_DATA1),keep_default_na=False,na_values=[''], encoding="cp1251",sep=';',header=None)
data_a1 = data_a1[[0,1,3,5,11,13,28,29]]
data_a1.columns = ['rack_cpu','chart','block','signal','value','link','type','out']
data_a1 = pd.concat([data_a1, data_a1['rack_cpu'].str.split("\\",expand = True)[[0,1]]], axis=1) 
data_a1.columns = ['rack_cpu','chart','block','signal','value','link','type','out','rack','cpu']
data_a1["out"]= data_a1["out"].replace(("IN","OUT"),(0,1)).astype('bool') 
data_a1 = data_a1[~data_a1['chart'].isin(IGNORE_CHART)]
data_a1["block"] = data_a1["block"].astype(str)
data_a1["chartblock"] = data_a1["chart"].str.upper()+"/"+data_a1["block"].str.upper()
print("Data types ",data_a1['type'].unique())
print("Charts ",data_a1['chart'].unique())
data_a1.head()

Data types  ['BOOL' 'REAL' 'SDTIME' 'WORD' 'INT' 'DINT' 'GLOBAL' 'STRING' 'DWORD'
 'BYTE']
Charts  ['ENCC_1' 'ENBRU1' 'ENBRI1' 'ENBEA1' 'BGT__1' '@SEND1' 'ENGAU1' 'ENGN11'
 'ENMB_1' 'ENOSW1' 'ENGUA1' 'ENHDR1' 'ENIHW1' 'ENINS1' 'ENEXT1' 'ENENT1'
 'ENSR_1' 'ENSPH1' 'ENSCD1' 'ENPIN1' 'ENPEL1' 'ENPAR1' 'ENISW1' 'ENLAP1'
 'ENMA_1' 'ENMAN1' 'ENOHW1' 'ENSTR1' 'ENTEM1' 'ENTUR1' 'MESY_1' 'MONIM1'
 'MONIW1' 'SERV_1' 'VISU_1' 'WPS__1' 'ENISW_MSU' 'ENDIA1' 'ENCRA1'
 'ENCPT1' 'ENCPS1' 'ENCPP1' 'ENCPM1' 'ENCPH1' 'ENCPC1' 'ENCPA1' 'ENCOO1'
 'ENMG_1' 'ENMC_1' 'ENMD_1' 'ENME_1' 'ENMF_1' '@SEND2' 'EXBAN2' 'EXBBH2'
 'EXBBY2' 'EXBEA2' 'EXBW_2' 'EXBWA2' 'EXBWT2' 'EXCC_2' 'EXCRA2' 'EXCRM2'
 'EXCUT2' 'EXDEF2' 'EXDIA2' 'EXFEL2' 'EXGAU2' 'EXGN12' 'EXGN22' 'EXIHW2'
 'EXIR_2' 'EXIRP2' 'EXISW2' 'EXKNA2' 'EXLAP2' 'EXMAN2' 'EXMLR2' 'EXMLT2'
 'EXOHW2' 'EXOSW2' 'EXPAR2' 'EXPUS2' 'EXREJ2' 'EXREL2' 'EXSC_2' 'EXSCA2'
 'EXSCM2' 'EXSH12' 'EXSH22' 'EXSHJ2' 'EXSHT2' 'EXSML2' 'EXSR_2' 'EXSRC2'
 'EXSRE2' 'EXSSC2' 'EXSSR2' 'EX

,rack_cpu,chart,block,signal,value,link,type,out,rack,cpu,chartblock
0,TX04\D01P01\,ENCC_1,ABLOKD,I1,NaN,ENCC_1\AND209.Q,BOOL,False,TX04,D01P01,ENCC_1/ABLOKD
1,TX04\D01P01\,ENCC_1,ABLOKD,I2,NaN,ENCC_1\NOTBDW.Q,BOOL,False,TX04,D01P01,ENCC_1/ABLOKD
2,TX04\D01P01\,ENCC_1,ABLOKD,Q,0,ENCC_1\XN1030.I4,BOOL,True,TX04,D01P01,ENCC_1/ABLOKD
3,TX04\D01P01\,ENCC_1,ADD100,X1,NaN,ENCC_1\DIV001.Y,REAL,False,TX04,D01P01,ENCC_1/ADD100
4,TX04\D01P01\,ENCC_1,ADD100,X2,NaN,ENCC_1\CTB003.Y15,REAL,False,TX04,D01P01,ENCC_1/ADD100


In [152]:
#Generate Simadyn chart structure
def sd_get_chart_list(PATH_TO_STRUCG=PATH_TO_STRUCG,SD_RACK_NAMES=SD_RACK_NAMES):
    #filter chart files
    n = re.compile("^(((?!old).)*).((cfp)|(ofp))$")
    file_list=[] 
    file_list = {rack_name:[item for item in os.listdir(os.path.join(PATH_TO_STRUCG,rack_name)) \
        # chart file names filter conditions
        if os.path.isfile(os.path.join(PATH_TO_STRUCG,rack_name, item)) and (n.search(item)) and len(item)<11 \
        ] for rack_name in SD_RACK_NAMES}
    file_list_filtered = dict(file_list)
    for i in file_list:
        for k in file_list[i]:
            chart_name = k.split('.')[0]
            if "%s.cfp"%chart_name in file_list[i]:
                try:
                    file_list_filtered[i].remove("%s.ofp"%chart_name)
                except ValueError:
                    pass  # do nothing
    out_dict = dict()
    for i in file_list_filtered:
        out_dict[i] = dict()
        for k in file_list_filtered[i]:
            out_dict[i][k.split('.')[0]]=os.path.join(PATH_TO_STRUCG,i,k)
    if out_dict:
        return out_dict
    else:
        return -1
    
#Generate Simadyn cpu mapping structure
def sd_get_cpu_charts_list(PATH_TO_STRUCG=PATH_TO_STRUCG,SD_RACK_NAMES=SD_RACK_NAMES):
    #filter chart files
    n = re.compile("^((\d+).*)\.(mpn)$")
    file_list=[] 
    file_list=[] 
    file_list = {rack_name:[item for item in os.listdir(os.path.join(PATH_TO_STRUCG,rack_name)) \
        # chart file names filter conditions
        if os.path.isfile(os.path.join(PATH_TO_STRUCG,rack_name, item)) and (n.search(item)) and len(item)<11 \
        ] for rack_name in SD_RACK_NAMES}
    out_dict = dict()
    for i in file_list:
        out_dict[i] = dict()
        for k in file_list[i]:
            out_dict[i][k.split('.')[0]]=os.path.join(PATH_TO_STRUCG,i,k)
    if out_dict:
        return out_dict
    else:
        return -1
    
sd_chart_list = sd_get_chart_list() 
sd_cpu_charts_list = sd_get_cpu_charts_list() 

##output => #sd_chart_list {"tx03":{'chrt_name':file_addrs}}

{'tx04': {'1tx04': '/notebooks/pinda/data/test/tx04/1tx04.mpn',
  '2tx04': '/notebooks/pinda/data/test/tx04/2tx04.mpn',
  '3tx04': '/notebooks/pinda/data/test/tx04/3tx04.mpn'}}

In [11]:
#n = re.search("^(((?!old).)*).((cfp)|(ofp))$", 'pray1___20131208.cfp')
test_txt = []
test_txt.append(" 325	I1  B1 < VIHW09.Q15&")
test_txt.append('  81	IS  V2 < $VISW17')
test_txt.append(" 163	Q1  B1 > ,'Man__CPS'            \"MAINTENANCE COIL PREP STATION\"")
test_txt.append(' 115	I1  B1 < VIHW17.Q2&')
test_txt.append(' 519	X1  N2 < @TYP=V2,0HFFFF')
test_txt.append(' 520	X2  N2 < @TYP=V2,LB2.QS&')
test_txt.append(' 527	QS  V2 > $VGAU02                "Diagnose 2nd direction"')
test_txt.append(' 528	Q   B1 > ')
test_txt.append("  31	AD  NK = D02_I1.X8C")
test_txt.append("  32	DM  B1 - 1")
test_txt.append("  22	AR  NS - 'B1G1X1'")
test_txt.append("  21	CTS CR - D0300A")
test_txt.append("1151	T1  NS - '@F@85@ EN QFC Coil P. Shear Tab. Air Float'")
test_txt.append("1199	X1  N2 < 0%")
test_txt.append("1200	X2  N2 < SEND07.QS")
test_txt.append("1201	I   B1 < EN_MES.Q,'EN_MES'&")
test_txt.append(" 566	LZU NF < #10000[ms],'RunTime '  \"Run time monitoring\"")
test_txt.append("1207	X2  N2 < 18.298%                \"18,3 representa 3000 kg\"")
test_txt.append("1283	X2  NF < #0")
test_txt.append("1304	X2  NF < #26,'GRAUS'")
test_txt.append(" 905	X2  NF < #100.0000000E3")
test_txt.append("1074	Y   NF > @TYP=TF,$PTTI_A,'PTTI_AUT'&")
test_txt.append("1073	X   NF < @TYP=TF,10[S]")
test_txt.append("2983	X2  NF < #530.0000000E-3,'m'")
test_txt.append(" 439	FBU NF < #500[ms],'Feedback'    \"Feedback monitoring\"")
test_txt.append(" 494	I   B1 < STG641.Q")
test_txt.append(" 495	Y   N2 > @TYP=V2")
test_txt.append("  17	X1  NF < $XIHW05,SCAL=1[m]")
test_txt.append(" 374	T   TF <  3           [s]")
test_txt.append(" 169	T   TF <            2[s ]")
test_txt.append("2946	OR  V2 < 0B00000000 00000000")
test_txt.append(" 108	X05 NF < #2500.0[V/(m/s)],'ky'  \"Scaling factor for control output\"&")
test_txt.append("2756	CRT TR = !EXM504")
test_txt.append(" 337	Q1  B1 >")
test_txt.append("1407	T   TF < #2.5 [s]")
test_txt.append("  87	Q3  B1 > $E_STOP PN,'E_STOP'&")

test_txt2 = " 439	FBU NF < #500[ms],'Feedback'    \"Feedback monitoring\""


#m = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>])\s+(@TYP=(..),)?([^@\"',&]+)([,\"'&]|$)", )
m = re.compile(r"""
        ^\s*\d+\s+ #string number
        (\w+)\s+   #signal name 
        (\w+)\s+   #signal type 
        ([<>=-])\s*  #signal delimiter 
        (@TYP=(..),?)? #singal type
        (   #value
        ([a-zA-Z0-9_.]+)|  #can be connection to other block
        ([$%!\[\]/()a-zA-Z0-9_.#-]+)|  #can be value
        ([$a-zA-Z0-9_]+\s*[$a-zA-Z0-9_]*)|  #can be virtual connection
        ([#0-9.]+\s*\[.*\])|  #can be time with spaces
        ([0-9B]+\s+[0-9]+)|  #can be hex in bit representation
        ('.*') #can be text
        )? #end of value
        (,)? #divider
        ('.*')? #link comment
        (,.*)? #some comment
        (\s+".*")? #signal comment
        (&)? #check next string
        ($) #end of string
        """, re.X)
k = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>=-])")

for a in test_txt:
    n = m.search(a)
    print(a)
    #print(n.group(1),n.group(2),n.group(3),n.group(5),n.group(6),n.group(13))
    print(n.group(6))
    #print(n)
    
#n = m.search(test_txt2)
#print("OUT:",n.group(1),n.group(2),n.group(3),n.group(6),n.group(13))

#for a in test_txt:
#    n = k.search(a)
#    print(n)

#for a in test_txt:
#    n = k.search(a)
#    print(n)

 325	I1  B1 < VIHW09.Q15&
VIHW09.Q15
  81	IS  V2 < $VISW17
$VISW17
 163	Q1  B1 > ,'Man__CPS'            "MAINTENANCE COIL PREP STATION"
None
 115	I1  B1 < VIHW17.Q2&
VIHW17.Q2
 519	X1  N2 < @TYP=V2,0HFFFF
0HFFFF
 520	X2  N2 < @TYP=V2,LB2.QS&
LB2.QS
 527	QS  V2 > $VGAU02                "Diagnose 2nd direction"
$VGAU02
 528	Q   B1 > 
None
  31	AD  NK = D02_I1.X8C
D02_I1.X8C
  32	DM  B1 - 1
1
  22	AR  NS - 'B1G1X1'
'B1G1X1'
  21	CTS CR - D0300A
D0300A
1151	T1  NS - '@F@85@ EN QFC Coil P. Shear Tab. Air Float'
'@F@85@ EN QFC Coil P. Shear Tab. Air Float'
1199	X1  N2 < 0%
0%
1200	X2  N2 < SEND07.QS
SEND07.QS
1201	I   B1 < EN_MES.Q,'EN_MES'&
EN_MES.Q
 566	LZU NF < #10000[ms],'RunTime '  "Run time monitoring"
#10000[ms]
1207	X2  N2 < 18.298%                "18,3 representa 3000 kg"
18.298%
1283	X2  NF < #0
#0
1304	X2  NF < #26,'GRAUS'
#26
 905	X2  NF < #100.0000000E3
#100.0000000E3
1074	Y   NF > @TYP=TF,$PTTI_A,'PTTI_AUT'&
$PTTI_A
1073	X   NF < @TYP=TF,10[S]
10[S]
2983	X2  NF < #530.0000000E-

In [224]:
#test_txt="   &	V4.2.6 FP-COP CHK 22.02.;9 07:54  (MP-TX04.PN-D05_P2 13.09.;6 05:02)"
#test_txt = "   1 PN-D01_P1"
#test_txt = "1832	*"
#test_txt = ""
#test_txt = "  89 Connector list of the function packets"
#test_txt = "1816	FP-@SEND1,COM"
test_txt = "1679	   $VINS02 T4 < PN     PN-D01_P1"

#cp = re.compile(r'^\s+\d+\s+([a-zA-Z0-9_-]+)$')
#cp = re.compile(r'^\s+\d+\s+Connector list of the function packets')
#cp = re.compile(r'^\s*\d+\s+\*')
#cp = re.compile(r'^\s*\d+\s+FP-([a-zA-Z0-9_\-@]+)')
cp = re.compile(r'^\s*\d+\s+(\$[a-zA-Z0-9_\-@]+)\s([\w]+)\s((<)|(>))\s([\w]+)\s+([a-zA-Z0-9_-]+)$')
cp_s = cp.search(test_txt)
print(cp_s)
print(cp_s.group(1),cp_s.group(3),cp_s.group(7))

<_sre.SRE_Match object; span=(0, 37), match='1679\t   $VINS02 T4 < PN     PN-D01_P1'>
$VINS02 < PN-D01_P1


In [123]:
test_txt = []
test_txt.append("   &		(SEND01.I9)")
test_txt.append('   &		(KA1014.I2,KA1015.I2)')
k = re.compile(r"^\s*&+\s+\((.*)\)\s*$")
for a in test_txt:
    n = k.search(a)
    print(n.group(1))
    #print(n)

SEND01.I9
KA1014.I2,KA1015.I2


In [326]:
#########################row data generating library#################

#open chart for reading
def sd_open_chart(chart_addr):
    try:
        with open(chart_addr) as f:
            lines = [line.rstrip('\n') for line in f]
    except EnvironmentError: # parent of IOError, OSError *and* WindowsError where available
        #print('ERROR File not exists!')
        return -1
    return lines

#generate chart mapping: line {block name:[block line begin, block line end]}
def sd_get_chart_map(inp_line):
    b = re.compile(r"^\s*\d+ ?([a-zA-Z0-9_]+) +: [a-zA-Z0-9_.@]+\s*,\s*POS=")
    el = re.compile(r'^ *\d+\s+[+]')
    #cp = re.compile(r'\(([a-zA-Z0-9_-]+).([a-zA-Z0-9_-]+)\s+\S+\s+\S+\)') #not used, because not all charts consist
    block_list = []
    mark_bloc = False
    #mark_cpu = False
    out_dict = dict()
    line_counter = 0
    for k in inp_line:
        #Find rack name and cpu name, e.g. MP-TX04 PN-D05_P2
        #if not mark_cpu:
        #    cp_s = cp.search(k)
        #    if cp_s :
        #        rack,cpu = cp_s.group(1),cp_s.group(2)
        #        mark_cpu = True
        b_s = b.search(k)
        if b_s :
            mark_bloc = True
            curr_block_name = b_s.group(1)
            block_list.append(curr_block_name)
            mark_bloc_line_beg = line_counter
        elif el.search(k):
            if mark_bloc:
                mark_bloc = False
                out_dict[curr_block_name] = [mark_bloc_line_beg,line_counter]
        line_counter = line_counter+1        
    #print(len(block_list))
    if(out_dict):
        return out_dict
    else:
        return -1

#check if line is signal or link
def sd_check_line_signal(line):
    sf = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>=-])")
    sa = re.compile(r"^\s*&+\s+(\(.*\))\s*$")
    if sf.search(line):
        out = 1 #signal
    elif sa.search(line):
        out = 2 #link
    else:
        out = 0 #nothing
    return out

#extract data from signal line
def sd_get_line_data(line): 
    m = re.compile(r"""
        ^\s*\d+\s+ #string number
        (\w+)\s+   #signal name 
        (\w+)\s+   #signal type 
        ([<>=-])\s*  #signal delimiter 
        (@TYP=(..),?)? #singal type
        (   #value
        ([a-zA-Z0-9_.]+)|  #can be connection to other block
        ([$%!\[\]/()a-zA-Z0-9_.#-]+)|  #can be value
        ([$a-zA-Z0-9_]+\s*[$a-zA-Z0-9_]*)|  #can be virtual connection
        ([#0-9.]+\s*\[.*\])|  #can be time with spaces
        ([0-9B]+\s+[0-9]+)|  #can be hex in bit representation
        ('.*') #can be text
        )? #end of value
        (,)? #divider
        ('.*')? #link comment
        (,.*)? #some comment
        (\s+".*")? #signal comment
        (&)? #check next string
        ($) #end of string
        """, re.X)
    #print(line)
    n = m.search(line)
    try: 
        out = list([n.group(1),n.group(2),n.group(3),n.group(5),n.group(6)])
    except:
        print(line)
        raise
        
                 #name      type       dir        type_conv  value      
    return list([n.group(1),n.group(2),n.group(3),n.group(5),n.group(6)])

#exstract link from like
def sd_get_line_link(line): 
    m = re.compile(r"^\s*&+\s+\((.*)\)\s*$")
    n = m.search(line)     
    return n.group(1)  

#get block data: signals, values, links, data types...
def sd_get_block_data(lines):
    NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE = 0,1,2,3,4
    mark_signal_row_name = None
    mark_signal_row_data = list([[],[]]) #[[signal data],[links]]
    mark_block_row_data = []
    count = 0;
    for l in lines:
        line_check_result = sd_check_line_signal(l)
        if line_check_result:
            if line_check_result==1:  
                #if found a signal line
                count = count+1
                #add previous signal to preparation table
                if mark_signal_row_name:
                    #print("Old name:",mark_signal_row_name)
                    mark_block_row_data.append(mark_signal_row_data)
                    mark_signal_row_data = list([[],[]]) #reset buffer table
                #name,type,dir,type_conv,more
                mark_signal_row_data[0] = sd_get_line_data(l)
                mark_signal_row_name = mark_signal_row_data[0][NAME]
                #print(mark_signal_row_name)
            elif line_check_result==2:
                #if found a link line
                mark_signal_row_data[1].extend(sd_get_line_link(l).split(','))
            #print(mark_signal_row_data)            
        else:
            pass #skip string
    #Add last signal to preparation storage
    mark_block_row_data.append(mark_signal_row_data)
    return mark_block_row_data

#convert list to pandas data frame
def sd_conv_block_data_to_df(block_data):
    #list element of row signal data:[[NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE],[links]]
    block_df = pd.DataFrame([[j for j in k[0]]+[k[1]] for k in block_data])
    return block_df
#convert list to better representation
def sd_conv_block_data_to_list(block_data):
    #list element of row signal data:[[NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE],[links]]
    block_list = [[j for j in k[0]]+[k[1]] for k in block_data]
    return block_list


#create df row with link source structure: chart_block_signal,cpu_link
def sd_get_source_link(block_data_df):
    m = re.compile(r"^\$\w+")
    out_df = []
    for i, j in block_data_df.iterrows():
        if j[2]==">":
            #look for source link in value field
            if(j[4]):
                n = m.search(j[4]) 
                if n:
                    chart_block_signal = j[6].upper()+'/'+j[7].upper()+'.'+j[0].upper()
                    cpu_link = j[8]+'_'+j[4]
                    #print(chart_block_signal,"=>",cpu_link)
                    out_df.append([chart_block_signal,cpu_link])
            #look for source link in link field
            for k in j[5]:
                if(k):
                    n = m.search(k) 
                    if n:
                        chart_block_signal = j[6].upper()+'/'+j[7].upper()+'.'+j[0].upper()
                        cpu_link = j[8]+'_'+k
                        #print(chart_block_signal,"=>",cpu_link)
                        out_df.append([chart_block_signal,cpu_link])
    return out_df

#generate data of all charts and generate data of link sources
def sd_build_data_s1(sd_chart_list,data_link_map_df):
    data_df = pd.DataFrame(None, index=[], columns=range(8))
    data_list = []
    data_link_source_df = pd.DataFrame(None, index=[], columns=['chart_block_signal','cpu_link'])
    for i in sd_chart_list:
        for (k,v) in sd_chart_list[i].items(): 
            chart = k.upper()
            chart_lines = sd_open_chart(v)
            chart_mapping = sd_get_chart_map(chart_lines)
            print("Parsing '%s' chart with %s functions"%(chart,len(chart_mapping)))
            for block_map in chart_mapping:
                #print(block_map,chart_mapping[block_map])
                block_lines = chart_lines[chart_mapping[block_map][0]:chart_mapping[block_map][1]]
                block_data = sd_get_block_data(block_lines)
                ##block_data_df = sd_conv_block_data_to_df(block_data)
                block_data_list = sd_conv_block_data_to_list(block_data)
                #add chart name
                ##block_data_df[6] = chart
                block_data_list.append(chart)
                #add block name
                ##block_data_df[7] = block_map
                block_data_list.append(block_map)
                ##block_data_df[8] = sd_get_chart_cpu(data_link_map_df,chart)
                block_data_list.append(sd_get_chart_cpu)
                #append block array to full array
                ##data_df = data_df.append(block_data_df, ignore_index = True) 
                data_list = 
                #create link source array
                block_link_source_arr = sd_get_source_link(block_data_df)
                if len(block_link_source_arr)!=0:
                    data_link_source_df = data_link_source_df.append(block_link_source_arr, ignore_index = True) 
                    
    return data_df,data_link_source_df

#generate cpu,link db by cpu mapping file
def sd_build_cpu_map(sd_chart_list):
    data_link_map_list = []
    cp = re.compile(r'^\s+\d+\s+([a-zA-Z0-9_-]+)$') #find CPU name in file
    bg = re.compile(r'^\s+\d+\s+Connector list of the function packets') # find beginning of link information
    en = re.compile(r'^\s*\d+\s+\*')
    ch = re.compile(r'^\s*\d+\s+FP-([a-zA-Z0-9_\-@]+)')
    lk = re.compile(r'^\s*\d+\s+(\$[a-zA-Z0-9_\-@]+)\s([\w]+)\s((<)|(>))\s([\w]+)\s+([a-zA-Z0-9_-]+)$')
    
    for i in sd_chart_list:
        for (k,v) in sd_chart_list[i].items():
            chart_lines = sd_open_chart(v)
            print("Parsing '%s' chart "%(k))
            cpu_mark = False
            beg_mark = False
            end_mark = False
            for l in chart_lines:
                ##Find CPU name
                if not cpu_mark:
                    cp_s = cp.search(l)
                    if cp_s :
                        cpu = cp_s.group(1)
                        cpu_mark = True
                        print(cpu)   
                else:
                    if not beg_mark:
                        if bg.search(l):
                            beg_mark = True
                    else:
                        #check end
                        if en.search(l):
                            end_mark = True
                            break
                        else:
                            #main block:
                            ch_s = ch.search(l)
                            if ch.search(l): #if chart beginning 1739	FP-MONIW1
                                chart = ch_s.group(1)
                            else:
                                lk_s = lk.search(l)
                                if lk_s: #if chart link definition 1744	   $DMALB1 T5 < PN     PN-D01_P1
                                    #'cpu','chart','out','link','cpu_source'
                                    data_link_map_list.append([cpu, chart.upper(),lk_s.group(3),lk_s.group(1),lk_s.group(7)])
    data_link_map_df = pd.DataFrame(data_link_map_list, columns=['cpu','chart','out','link','cpu_source'])
    return data_link_map_df

def sd_get_chart_cpu(data_link_map_df,chart):
    return data_link_map_df[data_link_map_df['chart']==chart].iloc[0]['cpu']
    

In [327]:
#execution cell
#creating row data DataFrame "data_s1" from chart list
        
data_lm = sd_build_cpu_map(sd_cpu_charts_list)
data_s1,data_ls1 = sd_build_data_s1(sd_chart_list,data_lm)
data_s1.columns = ['signal','type_base','inout','type_conv','value','link','chart','block','cpu']
data_s1["block"] = data_s1["block"].astype(str)

#save intermidiate result
#data_s1.to_csv(os.path.join(PATH_TO_DATA,FILE_NAME_TEMP1))

data_ls1


Parsing '1tx04' chart 
PN-D01_P1
Parsing '2tx04' chart 
PN-D05_P2
Parsing '3tx04' chart 
PN-D08_P3
Parsing 'ENBRI1' chart with 70 functions
Parsing 'ENBRU1' chart with 81 functions
Parsing 'ENCC_1' chart with 586 functions


KeyboardInterrupt: 

In [8]:
#restore intermediate result
data_s1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_NAME_TEMP1))


In [9]:
#rectify function: move Links "AAA.AA" of inputs to row "link"
def sd_rect_data_s1(data_df):
    def filter_func_output(x):
        if(re.search(r'(\S+[.]\D+)',str(x[4]))):
            x[5]=[x[4]]
        return x
    return data_df.apply(filter_func_output, axis = 1)

#rectify function: add block "chartblock"
def sd_rect_data_s2(data_df):
    data_df['chartblock'] = data_df["chart"].str.upper()+"/"+data_df["block"].str.upper()
    return data_df



In [10]:
data_s1 = sd_rect_data_s1(data_s1)

In [11]:
data_s2 = sd_rect_data_s2(data_s1)

In [35]:
test_df = data_s2.copy()
test_df[test_df['value'].apply(lambda x: bool(re.search(r"\$\w+\s+\w+",str(x)))==True)]

,Unnamed: 0,signal,type_base,inout,type_conv,value,link,chart,block,chartblock
71787,71787,Q3,B1,>,NaN,$E_STOP PN,[],enihw1,BII001,ENIHW1/BII001
101304,101304,CDM,B1,>,NaN,$BCSCDM PN,[],@send1,CS22,@SEND1/CS22


In [36]:

#report generator of comparison of block in all SD and TDC charts
def report_find_miss_blocks (data_sd,data_tdc, to_file = True, timestamp = datetime.datetime.now()):
    README_HEADER = COMPARATOR_README +"\
\r\nReport: find missing blocks in TDC program\
\r\nDate ant time of report generation: %s \
\r\n===================================================================================\
\r\n"%(str(timestamp))
    
    def generate_report(f):
        print(README_HEADER,file = f) 
        sd_uniq_blocks = data_sd['chartblock'].unique()
        tdc_uniq_blocks = data_tdc['chartblock'].unique()
        for i in sd_uniq_blocks:
            if not i in tdc_uniq_blocks:
                print("Missing block in TDC: ",i,"\r\n",file = f) 
        
        
        
    if to_file:
        file_suffix = "_find_miss_blocks_sd"
        file_prefix = "report_"
        try:
            # Create target Directory
            os.mkdir(os.path.join(PATH_TO_REPORTS))
        except FileExistsError:
            pass
        #sys.stdout = open(os.path.join(PATH_TO_REPORTS,(file_prefix+FILE_NAME_DATA1.split(".")[0]+file_suffix+".txt")), 'w')
        with open(os.path.join(PATH_TO_REPORTS,(file_prefix+FILE_NAME_DATA1.split(".")[0]+file_suffix+".txt")), 'w') as f: 
            generate_report(f)
    else:
        generate_report(sys.stdout)

        #data_a1_1 = data_a1
        #for i, j in data_a1_1.iterrows():
        #    if(not j['out']) and not pd.isnull(j['value']):

    
    
#test_df = data_s2.head(5000).copy()


In [40]:
#############EXECUTION GENERATE REPORT report_find_miss_blocks############
report_find_miss_blocks(data_s2,data_a1,to_file = True)